In [2]:
import pandas as pd
df=pd.read_csv("twitter_training.csv",header=None,
names=["tweet_id","entity","sentiment","text"])

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   74682 non-null  int64 
 1   entity     74682 non-null  object
 2   sentiment  74682 non-null  object
 3   text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
df.isnull().sum().sum()

np.int64(686)

In [6]:
df.head()

,tweet_id,entity,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [7]:
df.fillna(0,inplace=True)

In [25]:

valid = ["Negative", "Neutral", "Positive"]
df= df[df["sentiment"].isin(valid)]

In [26]:
sent_map = {"Negative":0,"Neutral":1,"Positive":2}
df["label"]= df["sentiment"].map(sent_map)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

In [28]:
X_train = X_train.astype(str)
X_test = X_test.astype(str)

In [29]:
X_train = X_train.fillna("").astype(str)
X_test = X_test.fillna("").astype(str)

In [30]:
tokenizer = Tokenizer(num_words=20000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


In [31]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [32]:
max_len = 40
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post")

In [33]:
df.head()

,tweet_id,entity,sentiment,text,label
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,2
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,2
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,2
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,2
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,2


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model = Sequential([
    Embedding(input_dim=20000, output_dim=64, input_length=max_len),
    SimpleRNN(64, return_sequences=False),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")
])

C:\Users\sudha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [35]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [36]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [37]:

# Explicitly build the model before printing the summary
model.build(input_shape=(None, max_len))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 40, 64)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,290,435 (4.92 MB)

 Trainable params: 1,290,435 (4.92 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64
)


loss, acc = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", acc)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.9612 - loss: 0.0863 - val_accuracy: 0.8297 - val_loss: 0.6400
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.9649 - loss: 0.0780 - val_accuracy: 0.8307 - val_loss: 0.6595
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.9624 - loss: 0.0832 - val_accuracy: 0.8141 - val_loss: 0.6462
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.9665 - loss: 0.0732 - val_accuracy: 0.8241 - val_loss: 0.6357
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.9658 - loss: 0.0769 - val_accuracy: 0.8328 - val_loss: 0.6565
386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8313 - loss: 0.6491
Test Accuracy: 0.8312667012214661


In [42]:
import numpy as np

In [43]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test_pad)
y_pred_labels = np.argmax(y_pred, axis=1)

386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [44]:
print(classification_report(y_test, y_pred_labels))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      4509
           1       0.85      0.80      0.82      3650
           2       0.82      0.87      0.84      4180

    accuracy                           0.83     12339
   macro avg       0.83      0.83      0.83     12339
weighted avg       0.83      0.83      0.83     12339



In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense,LSTM


In [46]:
model = Sequential([
    Embedding(input_dim=20000, output_dim=64, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")
])

C:\Users\sudha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [47]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [48]:
# Explicitly build the model before printing the summary
model.build(input_shape=(None, max_len))
model.summary()

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64
)

loss, acc = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", acc)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 40, 64)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,315,203 (5.02 MB)

 Trainable params: 1,315,203 (5.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.5589 - loss: 0.8891 - val_accuracy: 0.7296 - val_loss: 0.7101
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 22s 36ms/step - accuracy: 0.8033 - loss: 0.5065 - val_accuracy: 0.8196 - val_loss: 0.4628
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - accuracy: 0.8796 - loss: 0.3227 - val_accuracy: 0.8445 - val_loss: 0.4254
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 22s 36ms/step - accuracy: 0.9044 - loss: 0.2424 - val_accuracy: 0.8596 - val_loss: 0.3810
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - accuracy: 0.9171 - loss: 0.2032 - val_accuracy: 0.8606 - val_loss: 0.3988
386/386 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8594 - loss: 0.3999
Test Accuracy: 0.8593889474868774


In [50]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test_pad)
y_pred_labels = np.argmax(y_pred, axis=1)

386/386 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


In [51]:
print(classification_report(y_test, y_pred_labels))

              precision    recall  f1-score   support

           0       0.86      0.89      0.88      4509
           1       0.89      0.79      0.84      3650
           2       0.84      0.88      0.86      4180

    accuracy                           0.86     12339
   macro avg       0.86      0.86      0.86     12339
weighted avg       0.86      0.86      0.86     12339



In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense,LSTM,Bidirectional

In [53]:
model = Sequential([
    Embedding(input_dim=20000, output_dim=64, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")
])

C:\Users\sudha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [54]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [55]:
# Explicitly build the model before printing the summary
model.build(input_shape=(None, max_len))
model.summary()

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64
)

loss, acc = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", acc)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 40, 64)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,350,275 (5.15 MB)

 Trainable params: 1,350,275 (5.15 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 54s 63ms/step - accuracy: 0.7004 - loss: 0.6866 - val_accuracy: 0.8119 - val_loss: 0.4936
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 40s 65ms/step - accuracy: 0.8742 - loss: 0.3264 - val_accuracy: 0.8602 - val_loss: 0.3690
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 36s 58ms/step - accuracy: 0.9188 - loss: 0.2066 - val_accuracy: 0.8801 - val_loss: 0.3274
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 36s 58ms/step - accuracy: 0.9370 - loss: 0.1541 - val_accuracy: 0.8856 - val_loss: 0.3345
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 36s 58ms/step - accuracy: 0.9465 - loss: 0.1266 - val_accuracy: 0.8857 - val_loss: 0.3490
386/386 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8809 - loss: 0.3384
Test Accuracy: 0.8809465765953064


In [56]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test_pad)
y_pred_labels = np.argmax(y_pred, axis=1)

386/386 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step


In [57]:
print(classification_report(y_test, y_pred_labels))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      4509
           1       0.90      0.86      0.88      3650
           2       0.90      0.86      0.88      4180

    accuracy                           0.88     12339
   macro avg       0.88      0.88      0.88     12339
weighted avg       0.88      0.88      0.88     12339



In [58]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense,LSTM,Bidirectional,GRU # great recurrent unit

In [59]:
model = Sequential([
    Embedding(input_dim=20000, output_dim=64, input_length=max_len),
    GRU(64, return_sequences=False),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")
])


C:\Users\sudha\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [60]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [61]:
# Explicitly build the model before printing the summary
model.build(input_shape=(None, max_len))
model.summary()

history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64
)

loss, acc = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", acc)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 40, 64)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,307,139 (4.99 MB)

 Trainable params: 1,307,139 (4.99 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 47s 64ms/step - accuracy: 0.5156 - loss: 0.9338 - val_accuracy: 0.7169 - val_loss: 0.6636
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 41s 67ms/step - accuracy: 0.8256 - loss: 0.4444 - val_accuracy: 0.8434 - val_loss: 0.3999
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.9015 - loss: 0.2494 - val_accuracy: 0.8661 - val_loss: 0.3618
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9269 - loss: 0.1780 - val_accuracy: 0.8720 - val_loss: 0.3572
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9400 - loss: 0.1414 - val_accuracy: 0.8755 - val_loss: 0.3725
386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8761 - loss: 0.3602
Test Accuracy: 0.8760839700698853


In [62]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test_pad)
y_pred_labels = np.argmax(y_pred, axis=1)

386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [63]:
print(classification_report(y_test, y_pred_labels))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88      4509
           1       0.87      0.87      0.87      3650
           2       0.91      0.85      0.88      4180

    accuracy                           0.88     12339
   macro avg       0.88      0.87      0.88     12339
weighted avg       0.88      0.88      0.88     12339

